In [ ]:
# 导入所需模块
from dataclasses import dataclass, field, asdict, astuple
from typing import List, Optional, Dict, Any
from datetime import datetime
import json

print("数据类环境设置完成！")


In [ ]:
# 基础数据类示例

# 1. 简单的数据类
@dataclass
class Person:
    """人员信息数据类"""
    name: str
    age: int
    email: str

# 2. 带默认值的数据类
@dataclass
class Product:
    """商品信息数据类"""
    name: str
    price: float
    quantity: int = 0
    category: str = "未分类"
    active: bool = True

# 3. 不可变数据类
@dataclass(frozen=True)
class Point:
    """不可变的点坐标"""
    x: float
    y: float
    
    def distance_to(self, other: 'Point') -> float:
        """计算到另一个点的距离"""
        return ((self.x - other.x) ** 2 + (self.y - other.y) ** 2) ** 0.5

# 演示基础数据类
print("=== 基础数据类演示 ===")

# 创建对象
person = Person(name="Alice", age=30, email="alice@example.com")
product = Product(name="笔记本电脑", price=5999.99, quantity=10)
point1 = Point(0.0, 0.0)
point2 = Point(3.0, 4.0)

print(f"人员信息: {person}")
print(f"商品信息: {product}")
print(f"点坐标: {point1}")
print(f"距离: {point1.distance_to(point2)}")

# 数据类的自动生成方法
print(f"\n对象相等性: {person == Person('Alice', 30, 'alice@example.com')}")
print(f"对象哈希: {hash(point1)}")  # 只有frozen=True的数据类才能哈希

# 尝试修改不可变对象（会抛出异常）
try:
    # 故意尝试修改不可变对象的属性来演示错误
    point1.x = 1.0  # type: ignore  # 这会抛出异常
except AttributeError as e:
    print(f"不可变对象修改失败: {e}")


In [ ]:
# 字段定义和默认值示例

# 1. 使用 field() 函数
@dataclass
class User:
    """用户信息数据类"""
    name: str
    age: int
    email: str = field(default="no-email@example.com")
    tags: List[str] = field(default_factory=list)
    created_at: datetime = field(default_factory=datetime.now)
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    # 排除某些字段不参与比较
    internal_id: int = field(default=0, compare=False)
    
    # 不包含在 repr 中的字段
    password_hash: str = field(default="", repr=False)

# 2. 后处理方法
@dataclass
class Temperature:
    """温度数据类"""
    celsius: float
    fahrenheit: float = field(init=False)  # 不在初始化中设置
    
    def __post_init__(self):
        """初始化后处理"""
        self.fahrenheit = self.celsius * 9/5 + 32

# 3. 继承和字段重写
@dataclass
class Employee(Person):
    """员工类，继承自 Person"""
    employee_id: str
    department: str
    salary: float = field(default=0.0, repr=False)
    
    def __post_init__(self):
        """验证员工信息"""
        if self.salary < 0:
            raise ValueError("薪资不能为负数")

# 4. 验证和转换
@dataclass
class BankAccount:
    """银行账户数据类"""
    account_number: str
    balance: float
    owner: str
    
    def __post_init__(self):
        """验证账户信息"""
        if len(self.account_number) != 10:
            raise ValueError("账户号码必须是10位")
        if self.balance < 0:
            raise ValueError("余额不能为负数")
        self.owner = self.owner.strip().title()

# 演示字段定义
print("=== 字段定义和默认值演示 ===")

# 创建用户
user1 = User(name="Alice", age=30)
user2 = User(name="Bob", age=25, email="bob@example.com", tags=["developer", "python"])

print(f"用户1: {user1}")
print(f"用户2: {user2}")

# 测试默认工厂
user1.tags.append("admin")
print(f"用户1标签: {user1.tags}")
print(f"用户2标签: {user2.tags}")  # 应该是空列表，证明默认工厂正常工作

# 温度转换
temp = Temperature(celsius=25.0)
print(f"温度: {temp.celsius}°C = {temp.fahrenheit}°F")

# 继承示例
employee = Employee(
    name="Charlie",
    age=35,
    email="charlie@company.com",
    employee_id="E001",
    department="IT",
    salary=75000.0
)
print(f"员工: {employee}")

# 验证示例
try:
    account = BankAccount(
        account_number="1234567890",
        balance=1000.0,
        owner="  john doe  "
    )
    print(f"银行账户: {account}")
except ValueError as e:
    print(f"验证错误: {e}")

# 转换为字典和元组
user_dict = asdict(user1)
user_tuple = astuple(user1)
print(f"用户字典: {user_dict}")
print(f"用户元组: {user_tuple}")


In [ ]:
# 序列化和反序列化示例

from dataclasses import dataclass, field, asdict, astuple
from typing import List, Optional, Dict, Any
from datetime import datetime
import json

@dataclass
class Address:
    """地址数据类"""
    street: str
    city: str
    postal_code: str
    country: str = "中国"

@dataclass
class Contact:
    """联系人数据类"""
    name: str
    email: str
    phone: Optional[str] = None
    address: Optional[Address] = None
    tags: List[str] = field(default_factory=list)
    metadata: Dict[str, Any] = field(default_factory=dict)
    created_at: datetime = field(default_factory=datetime.now)
    
    def to_dict(self) -> Dict[str, Any]:
        """转换为字典，处理datetime序列化"""
        result = asdict(self)
        # 处理datetime序列化
        if isinstance(result['created_at'], datetime):
            result['created_at'] = result['created_at'].isoformat()
        return result
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'Contact':
        """从字典创建对象"""
        # 处理datetime反序列化
        if 'created_at' in data and isinstance(data['created_at'], str):
            data['created_at'] = datetime.fromisoformat(data['created_at'])
        
        # 处理嵌套对象
        if 'address' in data and data['address'] is not None:
            data['address'] = Address(**data['address'])
        
        return cls(**data)
    
    def to_json(self) -> str:
        """转换为JSON字符串"""
        return json.dumps(self.to_dict(), ensure_ascii=False, indent=2)
    
    @classmethod
    def from_json(cls, json_str: str) -> 'Contact':
        """从JSON字符串创建对象"""
        data = json.loads(json_str)
        return cls.from_dict(data)

# 自定义序列化器
class DataclassJSONEncoder(json.JSONEncoder):
    """自定义JSON编码器，处理数据类"""
    
    def default(self, o):
        if hasattr(o, '__dataclass_fields__'):
            # 如果是数据类，转换为字典
            return asdict(o)
        elif isinstance(o, datetime):
            # 处理datetime对象
            return o.isoformat()
        return super().default(o)

# 演示序列化和反序列化
print("=== 序列化和反序列化演示 ===")

# 创建复杂的数据对象
address = Address(
    street="中山路123号",
    city="深圳",
    postal_code="518000"
)

contact = Contact(
    name="张三",
    email="zhangsan@example.com",
    phone="13800138000",
    address=address,
    tags=["朋友", "同事"],
    metadata={"source": "手机通讯录", "priority": "high"}
)

print(f"原始联系人: {contact}")

# 转换为字典
contact_dict = contact.to_dict()
print(f"\n转换为字典: {contact_dict}")

# 转换为JSON
contact_json = contact.to_json()
print(f"\n转换为JSON:\n{contact_json}")

# 从JSON恢复
restored_contact = Contact.from_json(contact_json)
print(f"\n从JSON恢复: {restored_contact}")

# 验证数据一致性
print(f"\n数据一致性检查: {contact == restored_contact}")

# 使用自定义编码器
print("\n=== 自定义JSON编码器 ===")

# 创建包含多个联系人的列表
contacts = [
    contact,
    Contact(name="李四", email="lisi@example.com", tags=["客户"])
]

# 使用自定义编码器序列化
contacts_json = json.dumps(contacts, cls=DataclassJSONEncoder, ensure_ascii=False, indent=2)
print(f"联系人列表JSON:\n{contacts_json}")

# 批量处理示例
print("\n=== 批量处理示例 ===")

# 创建联系人列表
contact_data = [
    {"name": "王五", "email": "wangwu@example.com", "tags": ["朋友"]},
    {"name": "赵六", "email": "zhaoliu@example.com", "tags": ["同事"]},
    {"name": "孙七", "email": "sunqi@example.com", "tags": ["客户"]}
]

# 批量创建联系人对象
contacts_from_data = [Contact.from_dict(data) for data in contact_data]
print(f"批量创建的联系人数量: {len(contacts_from_data)}")

# 批量转换为字典
contacts_dicts = [asdict(contact) for contact in contacts_from_data]
print(f"批量转换的字典数量: {len(contacts_dicts)}")

# 保存到文件示例（模拟）
import tempfile
with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    json.dump(contacts_dicts, f, cls=DataclassJSONEncoder, ensure_ascii=False, indent=2)
    temp_file = f.name

print(f"数据已保存到临时文件: {temp_file}")

# 从文件加载（模拟）
with open(temp_file, 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)
    loaded_contacts = [Contact.from_dict(data) for data in loaded_data]

print(f"从文件加载的联系人数量: {len(loaded_contacts)}")

# 清理临时文件
import os
os.unlink(temp_file)
